In [ ]:
object_names = ['Switch', 'Oven', 'Bucket', 'Box', 'Toilet', 'KitchenPot', 'Microwave', 'Window', 'Kettle', 'Safe', 'Refrigerator', 'Table', 'StorageFurniture', 'Dishwasher', 'WashingMachine', 'FoldingChair', 'Phone', 'Door', 'Laptop', 'TrashCan', 'Stapler']
column_names = ['Model', 'Policy'] + object_names

In [ ]:
import pandas as pd
import os
import numpy as np
# models = ['flowbot', 'dit', 'pndit', 'hisdit', 'pnhisdit', 'hisditonly', 'hisonly', 'pndit&pn++', 'dit&pn++', 'largedit&pn++', 'flowbotallopen', 'pndit&pn++allopen>005','dit&pn++allopen>01', 'hispnditFilter']
models = ['flowbotMDrepeat5', 'flowbotROrepeat5', 'hispnditFilrepeat5', 'pnditrepeat5', 'his']
policies = ['raw', 'sgp', 'sgp+cc', 'sgp+cc+hf', 'sgp+hf']
# open_ratios = ['000_train', '001_train', '000_test', '001_test']
open_ratios = ['000_test', '000_train'] 

max_counts = {'000_train': {}, '001_train': {}, '000_test': {}, '001_test': {}}

success_rate_df = {
    '000_train': pd.DataFrame(np.zeros((7, len(object_names) + 2)), columns=column_names),
    '000_test': pd.DataFrame(np.zeros((7, len(object_names) + 2)), columns=column_names),
    '001_train': pd.DataFrame(np.zeros((7, len(object_names) + 2)), columns=column_names),
    '001_test': pd.DataFrame(np.zeros((7, len(object_names) + 2)), columns=column_names)
}
normalized_dist_df = {
    '000_train': pd.DataFrame(np.zeros((7, len(object_names) + 2)), columns=column_names),
    '001_train': pd.DataFrame(np.zeros((7, len(object_names) + 2)), columns=column_names),
    '000_test': pd.DataFrame(np.zeros((7, len(object_names) + 2)), columns=column_names),
    '001_test': pd.DataFrame(np.zeros((7, len(object_names) + 2)), columns=column_names),
}
for open_ratio_and_split in open_ratios:
    row_id = 0
    open_ratio, desired_split = open_ratio_and_split.split('_')[0], open_ratio_and_split.split('_')[1]
    for policy in policies:
        for model in models:
            path = f'./raw_tables/{model}_split_{policy}_{open_ratio}.csv'
            if not os.path.exists(path):
                continue
            result_df = pd.read_csv(path)

            srates = {}
            ndists = {}
            counts = {}
            for _, row in result_df.iterrows():
                name, split, sr, nd, count = row['index'].split('_')[0], row['index'].split('_')[1], row['success_rate'], row['norm_dist'], row['count']
                # print(name, sr, nd, count)
                # objects = list(set([name.split('_')[0] for name in result_df["index"].values]))
                # print(objects)
                if split != desired_split:  # Only include train / test
                    continue
                # name_split = '_'.join([name, split])
                if name not in counts.keys():
                    counts[name] = 0
                    srates[name] = 0
                    ndists[name] = 0
                # print(name, count)
                counts[name] += count
                srates[name] += sr * count
                ndists[name] += nd * count

            for category in counts.keys():
                if category not in max_counts[open_ratio_and_split].keys():
                    max_counts[open_ratio_and_split][category] = 0
                max_counts[open_ratio_and_split][category] = max(max_counts[open_ratio_and_split][category], counts[category])

            success_rate_df[open_ratio_and_split].at[row_id, 'Model'] = model
            normalized_dist_df[open_ratio_and_split].at[row_id, 'Model'] = model
            success_rate_df[open_ratio_and_split].at[row_id, 'Policy'] = policy
            normalized_dist_df[open_ratio_and_split].at[row_id, 'Policy'] = policy
            for category in counts.keys():
                success_rate_df[open_ratio_and_split].at[row_id, category] = srates[category]
                normalized_dist_df[open_ratio_and_split].at[row_id, category] = ndists[category]

            row_id += 1

    # Multiply by the max counts
    for id, row in success_rate_df[open_ratio_and_split].iterrows():
        for category in max_counts[open_ratio_and_split].keys():
            if max_counts[open_ratio_and_split][category] == 0:
                continue
            success_rate_df[open_ratio_and_split].at[id, category] /= max_counts[open_ratio_and_split][category]
            normalized_dist_df[open_ratio_and_split].at[id, category] /= max_counts[open_ratio_and_split][category]

            # Object count
            success_rate_df[open_ratio_and_split].at[row_id, category] = max_counts[open_ratio_and_split][category]
            normalized_dist_df[open_ratio_and_split].at[row_id, category] = max_counts[open_ratio_and_split][category]

In [ ]:
max_counts

In [ ]:
success_rate_df['000_train']

In [ ]:
success_rate_df['000_test']

In [ ]:
for key in open_ratios:
    success_rate_df[key].to_excel(f'./summary_success_rate_{key}.xlsx')
    normalized_dist_df[key].to_excel(f'./summary_norm_dist_{key}.xlsx')

In [ ]:
normalized_dist_df['001'].to_excel('./dit_norm_dist_summary_001.xlsx')